<a href="https://colab.research.google.com/github/GauthamBellamkonda/DigitRecognizer/blob/main/CNN%20Model/src/DigitRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use this on Google Colab
# !pip install tensorflowjs

In [2]:
import datetime

In [3]:
# Use this on Google Colab
# %tensorflow_version 2.x 

import tensorflow as tf
import tensorflowjs as tfjs
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

2022-01-25 10:03:01.491630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-25 10:03:01.491651: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels
test_labels = test_labels

train_images = train_images.reshape(-1, 28 , 28, 1) / 255.0
test_images = test_images.reshape(-1, 28 , 28, 1) / 255.0

train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [7]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=1)

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = Sequential()
model.add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D())
model.add(Conv2D(64,kernel_size=5,activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
epochs=20
model.summary()
history = model.fit(train_images,train_labels, batch_size=60, epochs = epochs, 
                    validation_data = (test_images, test_labels), callbacks=[annealer, tensorboard_callback], verbose=1)


print("Accuracy: ", 100*max(history.history['val_accuracy']), "%")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

2022-01-25 10:53:38.932669: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.



Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
1000/1000 [==============================] - 16s 16ms/step - loss: 0.1383 - accuracy: 0.9592 - val_loss: 0.0389 - val_accuracy: 0.9872 - lr: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.00095.
Epoch 2/20
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0407 - accuracy: 0.9873 - val_loss: 0.0352 - val_accuracy: 0.9884 - lr: 9.5000e-04

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009025.
Epoch 3/20
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0286 - accuracy: 0.9908 - val_loss: 0.0252 - val_accuracy: 0.9920 - lr: 9.0250e-04

Epoch 00004: LearningRateScheduler setting learning rate to 0.000857375.
Epoch 4/20
1000/1000 [==============================] - 16s 16ms/step - loss: 0.0192 - accuracy: 0.9940 - val_loss: 0.0292 - val_accuracy: 0.9908 - lr: 8.5737e-04

Epoch 00005: LearningRateScheduler setting learning rate to 0.00081

In [6]:
tfjs.converters.save_keras_model(model, "./model")